In [24]:
import math
import random

In [25]:
with open('./input.txt') as file:
    projects = file.read().split()
projects

['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [26]:
# no of teachers 
thrs_c = 4
# = int(input('Enter no. of teachers '))
# no of students 
stds_c = 4
# int(input('Enter no. of students '))

In [27]:
stds = {}
thrs = {}
# limit of each teacher 
T_LIM = 2

In [28]:
# preferences of students to the projects 
for i in range(stds_c):
    idx = random.randint(0, len(projects) - 5)
    stds[i] = [projects[idx + j] for j in range(4)]
stds

{0: ['S2', 'S3', 'S4', 'S5'],
 1: ['S2', 'S3', 'S4', 'S5'],
 2: ['S1', 'S2', 'S3', 'S4'],
 3: ['S2', 'S3', 'S4', 'S5']}

In [29]:
 #//!Need to make it more random 
# preferences of teachers to the projects 
for i in range(thrs_c):
    idx = random.randint(0, len(projects) - 5)
    thrs[i] = [projects[idx + j] for j in range(4)]
thrs

{0: ['S2', 'S3', 'S4', 'S5'],
 1: ['S2', 'S3', 'S4', 'S5'],
 2: ['S2', 'S3', 'S4', 'S5'],
 3: ['S1', 'S2', 'S3', 'S4']}

In [30]:
print('Students - ')
print(stds)
print('Teachers - ')
print(thrs)

Students - 
{0: ['S2', 'S3', 'S4', 'S5'], 1: ['S2', 'S3', 'S4', 'S5'], 2: ['S1', 'S2', 'S3', 'S4'], 3: ['S2', 'S3', 'S4', 'S5']}
Teachers - 
{0: ['S2', 'S3', 'S4', 'S5'], 1: ['S2', 'S3', 'S4', 'S5'], 2: ['S2', 'S3', 'S4', 'S5'], 3: ['S1', 'S2', 'S3', 'S4']}


In [31]:
mapping = {i : idx for idx, i in enumerate(projects)}
r_mapping = {idx : i for i, idx in mapping.items()}
print("Normal mapping project -> number", mapping)
print("R mapping", r_mapping)
# r_mapping
# // basically here we are mapping project to their serial number 

Normal mapping {'S1': 0, 'S2': 1, 'S3': 2, 'S4': 3, 'S5': 4, 'S6': 5}
R mapping {0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'}


In [32]:
stlen = len(stds)
prlen = len(projects)
thrlen = len(thrs)
# stlen
# prlen
# thrlen

In [33]:
def check_constraints(matrix):
    # below forloop checks that each column should have only 1 cell with 1 else we will return false
    for i in matrix:
        count = 0
        for j in i:
            if j == 1:
                count += 1
        if count > 1:
            return False
    
    # finding rowlength and column length 
    rlen = len(matrix)
    clen = len(matrix[0])

    # now we are checking that each row should have only 1 cell with 1 else again return false 
    for column in range(clen):
        count = 0
        for row in range(rlen):
            if matrix[row][column] == 1:
                count += 1
        if count > 1:
            return False

    # now we re checking that the project assigned to the student is in his preference or not if yes return true else return false 
    for sidx, student in enumerate(matrix):
        for idx, choice in enumerate(student):
            if choice == 1:
                if not (r_mapping[idx] in  stds[sidx]):
                    return False

    return True

In [34]:
# assigning teacher to students via recursion
def recurse_call(teach, assign, level, ans_seq):
    res_seq = []
    # base case for last student ig
    if level == stlen - 1:
        for i in assign[level]:
            if teach[i] + 1 <= T_LIM:
                ans_seq.append(i)
                return ans_seq, True
        return ans_seq, False
    
    for i in assign[level]:
        if teach[i] + 1 <= T_LIM:
            teach[i] += 1
            res_seq, ans = recurse_call(teach.copy(), assign.copy(), level + 1, ans_seq)
            teach[i] -= 1
            if ans == True:
                res_seq.append(i)
                return res_seq,True
    return res_seq, False

In [35]:
# code to check teacher constraints like no teacher should get overburdened with more no of projects then they said 
def teach_constraint(matrix):
    assign = []
    for row in matrix:
        for cidx, col in enumerate(row):
            if col == 1:
                temp = []
                for teacher, subject in thrs.items():
                    if r_mapping[cidx] in subject:
                        temp.append(teacher)
        assign.append(temp)
    teach = [0 for i in range(thrlen)]
    ans = False
    res_seq = []
    for val in assign[0]:
        teach[val] = 1
        res_seq, ans = recurse_call(teach.copy(), assign.copy(), 1, [])
        teach[val] = 0
        if ans == True:
            res_seq.append(val)
            return res_seq, True
    return res_seq,False

In [36]:
# generating random matix 
#! Need to add check constraints like only teacher constraint is checked till now
def random_gen():
    matrix = [[0 for j in range(prlen)] for i in range(stlen)]
    booked = []
    t_idx = []

    while True:
        for i in range(stlen):
            idx = random.randint(0,len(projects) - 1)
            while not(r_mapping[idx] in stds[i]) or idx in booked:
                idx = random.randint(0,len(projects) - 1)
            matrix[i][idx] = 1
            booked.append(idx)
        res_seq, val = teach_constraint(matrix)
        if val:
            return matrix.copy()

    return matrix
        # if check_constraints(matrix):
        #     return matrix.copy()
        # else:
        #     for i in matrix:
        #         for j in range(len(i)):
        #             i[j] = 0

In [37]:
# constant for objective function
C = 12

In [38]:
# objective function as present in paper 
def obj_fun(matrix):
    val = 0
    for ridx, row in enumerate(matrix):
        for cidx, col in enumerate(row):
            if col == 1:
                val += stds[ridx].index(r_mapping[cidx]) + 1
    return val

In [39]:

def energy(matrix1, matrix2):
    val1 = obj_fun(matrix1)
    val2 = obj_fun(matrix2)
    return math.exp((val1 - val2)/C)

In [44]:
my_sequence = [random_gen()]
# my_sequence
print("Random sequence ", my_sequence)
c_energy = 0
max_energy = 0
idx = -1
for i in range(1000):
    idx -= 1
    temp = random_gen()
    ctr = 0
    # print(" This is temp ", temp)
    while temp in my_sequence and ctr < 9000:
        temp = random_gen()
        ctr += 1
    if ctr == 9000:
        break

    my_sequence.append(temp)
    c_energy = energy(my_sequence[-1], temp)
    
    if c_energy > max_energy:
        max_energy = c_energy
        idx = -1
    
for r_idx, row in enumerate(my_sequence[idx]):
    for c_idx, col in enumerate(row):
        if col == 1:
            print("Student", r_idx + 1, "-", r_mapping[c_idx])

ans_seq, val = teach_constraint(my_sequence[idx])
for idx, val in enumerate(ans_seq):
    print("Student", idx + 1, "assigned to teacher", val + 1)

Random sequence  [[[0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]]
 This is temp  [[0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
 This is temp  [[0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
 This is temp  [[0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
 This is temp  [[0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
 This is temp  [[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]
 This is temp  [[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
 This is temp  [[0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
 This is temp  [[0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
 This is temp  [[0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0]]
 This is temp  [[0, 0, 0, 1, 0, 0], 